## <center> Аттестация. Практическое задание в Jupiter

В данном блоке вам предстоит решить задачу, которую автор когда-то решал на одном из своих собеседований.
Вам предоставлена таблица с данными (скачать файл можно на платформе) о ежедневной выработке энергии солнечными батареями. Информация в ней содержится в двух столбцах: 
* date - дата наблюдения
* cum_power - накопленная (кумулятивная) выработка энергии на указанный день

То есть перед вами классический пример временного ряда: числовой показатель, зависящий от времени. Давайте с ним поработаем.


In [66]:
import pandas as pd
import numpy as np
import plotly
import plotly.express as px
print(plotly.__version__)
import json

5.4.0


In [67]:
solar_power = pd.read_csv('/home/tatiana/SolarPower/solarpower.csv')

In [68]:
solar_power.head(2)

,date,cum_power
0,2017-01-01,20111.0
1,2017-01-02,20112.0


In [69]:
solar_power.shape

(664, 2)

1. В нашем временном ряде есть несколько пропусков, они обозначены числом -1. Заполните пропуски средним арифметическим между двумя соседними наблюдениями: </p>
    $$x^*_i  = \frac{x_{i-1} - x_{i+1}}{2}$$ 
    
Совет: проще всего будет сделать это, используя цикл по индексам элементов столбца cum_power, но вы можете предложить более изящное решение.


In [70]:
s_power = solar_power.copy()

In [71]:
s_power[s_power['cum_power'] == -1]

,date,cum_power
45,2017-02-15,-1.0
647,2018-10-10,-1.0


In [72]:
for i in s_power['cum_power'].index:
    if s_power['cum_power'][i] == -1:
        s_power['cum_power'][i] = (s_power['cum_power'][i-1] - s_power['cum_power'][i+1]) / 2
    

/tmp/ipykernel_12317/2614581469.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



2. Теперь, когда таблица заполнена, создайте столбец day_power - ежедневный прирост выработки солнечной энергии.

    После чего создайте данные о ежемесячной выработке энергии на каждый год. 
    
Совет: для того, чтобы выделить из даты год и месяц проще всего воспользоваться методом аксессора dt.to_period('M'), который позволяет выделять периоды, в данном случае 'М' - месяц.

In [73]:
s_power['day_power'] = s_power['cum_power'].diff()
s_power = s_power.fillna(0)

In [74]:
s_power.head()

,date,cum_power,day_power
0,2017-01-01,20111.0,0.0
1,2017-01-02,20112.0,1.0
2,2017-01-03,20115.0,3.0
3,2017-01-04,20117.0,2.0
4,2017-01-05,20119.0,2.0


In [75]:
s_power['date'] = pd.to_datetime(s_power['date'])
s_power['date'] = s_power['date'].dt.to_period('M')
month_power = s_power.groupby('date',as_index=False).sum() 

In [76]:
month_power.head(2)

,date,cum_power,day_power
0,2017-01,624746.0,97.0
1,2017-02,546935.0,104.0


In [77]:
month_power.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22 entries, 0 to 21
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype    
---  ------     --------------  -----    
 0   date       22 non-null     period[M]
 1   cum_power  22 non-null     float64  
 2   day_power  22 non-null     float64  
dtypes: float64(2), period[M](1)
memory usage: 656.0 bytes


3. Постройте график временного ряда - ежемесячной выработки солнечной энергии. Не забудьте подписать оси и сам график.
Укажите в какие сезоны наблюдается подъем, а в какие спад?

In [78]:
fig = px.line(
    data_frame= month_power,
    x='date',
    y='day_power',
    height=500,
    width=1000,
    title='Ежемесячная выработака солнечной энергии',
    labels={'date':'Год и месяц','day_power':'Выработано энергии'}
)
fig.show()

TypeError: Object of type Period is not JSON serializable